In [1]:
import numpy as np
import cv2
import os
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans

In [28]:
def load_images_from_folder(folder):
    images = {}
    for filename in os.listdir(folder):
        category = []
        path = folder + "/" + filename
        for cat in os.listdir(path):
            img = cv2.imread(path + "/" + cat,0)
            if img is not None:
                category.append(img)
        images[filename] = category
    return images

images = load_images_from_folder('./train-gray') 
test = load_images_from_folder('./test-gray')

In [42]:
def sift_features(images):
    sift_vectors = {}
    descriptor_list = []
    sift = cv2.SIFT_create(nfeatures=40)
    for key,value in images.items():
        features = []
        for img in value:
            kp, des = sift.detectAndCompute(img,None)
           
            
            descriptor_list.extend(des)
            features.append(des)
        sift_vectors[key] = features
    return [descriptor_list, sift_vectors]

sifts = sift_features(images) 

descriptor_list = sifts[0] 

all_bovw_feature = sifts[1] 

test_bovw_feature = sift_features(test)[1] 

In [43]:
def kmeans(k, descriptor_list):
    kmeans = KMeans(n_clusters = k, n_init=10)
    kmeans.fit(descriptor_list)
    visual_words = kmeans.cluster_centers_ 
    return visual_words
       
visual_words = kmeans(150, descriptor_list) 

In [44]:
def find_index(image, center):
    count = 0
    ind = 0
    for i in range(len(center)):
        if(i == 0):
           count = distance.euclidean(image, center[i]) 
        else:
            dist = distance.euclidean(image, center[i]) 
            if(dist < count):
                ind = i
                count = dist
    return ind

In [45]:
def image_class(all_bovw, centers):
    dict_feature = {}
    for key,value in all_bovw.items():
        category = []
        for img in value:
            histogram = np.zeros(len(centers))
            for each_feature in img:
                ind = find_index(each_feature, centers)
                histogram[ind] += 1
            category.append(histogram)
        dict_feature[key] = category
    return dict_feature
   
bovw_train = image_class(all_bovw_feature, visual_words) 

bovw_test = image_class(test_bovw_feature, visual_words) 

In [46]:
def knn(images, tests):
    num_test = 0
    correct_predict = 0
    class_based = {}
    
    for test_key, test_val in tests.items():
        class_based[test_key] = [0, 0]
        for tst in test_val:
            predict_start = 0
            minimum = 0
            key = "a"
            for train_key, train_val in images.items():
                for train in train_val:
                    if(predict_start == 0):
                        minimum = distance.euclidean(tst, train)
                        key = train_key
                        predict_start += 1
                    else:
                        dist = distance.euclidean(tst, train)
                        if(dist < minimum):
                            minimum = dist
                            key = train_key
            
            if(test_key == key):
                correct_predict += 1
                class_based[test_key][0] += 1
            num_test += 1
            class_based[test_key][1] += 1
    return [num_test, correct_predict, class_based]
    
    
results_bowl = knn(bovw_train, bovw_test) 

In [47]:
 def accuracy(results):
    avg_accuracy = (results[1] / results[0]) * 100
    print("Average accuracy: %" + str(avg_accuracy))
    print("\nAccuracies based on activities: \n")
    for key,value in results[2].items():
        acc = (value[0] / value[1]) * 100
        print(key + " : %" + str(acc))
              
accuracy(results_bowl) 

Average accuracy: %33.68055555555556

Accuracies based on activities: 

bend : %50.0
jack : %34.0
jump : %36.84210526315789
pjump : %40.0
run : %45.45454545454545
side : %19.047619047619047
skip : %16.666666666666664
walk : %30.76923076923077
wave1 : %27.586206896551722
wave2 : %34.48275862068966
